# Imports

## The purpose of this notebook will be to provide an example of finding and showing the maximum common substructure between two ligands

In [ ]:
from asapdiscovery.data.schema_v2.ligand import Ligand
from pathlib import Path
from IPython.display import Image

# Import ligands

In [ ]:
mols = [Ligand.from_sdf(sdf) for sdf in Path("/Users/alexpayne/Scientific_Projects/mers-drug-discovery/sars2-retrospective-analysis/sdf_lsf_array_p_only_by_name_2d/").glob('*.sdf')]

In [ ]:
mol1 = mols[0]
mol2 = mols[1]

In [ ]:
from asapdiscovery.dataviz.molecules import display_openeye_ligand

In [ ]:
display_openeye_ligand(mol1.to_oemol())

In [ ]:
def display_openeye_ligand(mol, out_fn="test.png", aligned=False):
    from asapdiscovery.data.openeye import oedepict

    if not aligned:
        oedepict.OEPrepareDepiction(mol)
    disp = oedepict.OE2DMolDisplay(mol)
    clearbackground = False
    oedepict.OERenderMolecule(out_fn, disp, clearbackground)

In [ ]:
from rdkit import Chem, Geometry
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import MCS

# Function to load molecules from SMILES strings
def load_molecule(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return mol

# Function to find and plot maximum common substructure
def find_and_plot_mcss(mol1, mol2):
    # Generate Morgan fingerprints for each molecule
    fp1 = AllChem.GetMorganFingerprint(mol1, 2)
    fp2 = AllChem.GetMorganFingerprint(mol2, 2)

    # Find MCS
    mcs_result = MCS.FindMCS([mol1, mol2])
    mcs_smarts = mcs_result.smarts

    # Highlight MCS in the molecules
    pattern = Chem.MolFromSmarts(mcs_smarts)

    matching = [molecule.GetSubstructMatch(pattern) for molecule in [mol1, mol2]]
    molecules = [mol1, mol2]
    number = -1
    mol, match = molecules[0], matching[0]
    AllChem.Compute2DCoords(mol)
    coords = [mol.GetConformer().GetAtomPosition(x) for x in match]
    coords2D = [Geometry.Point2D(pt.x, pt.y) for pt in coords]
    for mol, match in zip(molecules, matching):
        if not match:
            continue
        coord_dict = {match[i]: coord for i, coord in enumerate(coords2D)}
        AllChem.Compute2DCoords(mol, coordMap=coord_dict)

    # Draw the molecules with the highlighted MCS
    legends = ["Mol1", "Mol2"]
    return Draw.MolsToGridImage(
        molecules,
        legends=legends,
        molsPerRow=5,
        highlightAtomLists=matching,
        subImgSize=(200, 200),
    )
    return img

In [ ]:
mol1_rdkit = load_molecule(mol1.smiles)
mol2_rdkit = load_molecule(mol2.smiles)

In [ ]:
find_and_plot_mcss(mol1_rdkit, mol2_rdkit)

In [ ]:
from asapdiscovery.data.openeye import oechem, oedepict
from openeye import oegraphsim

# Function to load molecules from SMILES strings
def load_molecule(smiles):
    mol = oechem.OEGraphMol()
    oechem.OESmilesToMol(mol, smiles)
    return mol

# Function to find and plot maximum common substructure
def find_and_plot_mcss(mol1, mol2):
    
    # Find MCS
    atomexpr = oechem.OEExprOpts_AutomorphAtoms
    bondexpr = (
        oechem.OEExprOpts_Aromaticity
        | oechem.OEExprOpts_BondOrder
        | oechem.OEExprOpts_RingMember
    )

    pattern_query = oechem.OEQMol(mol1.to_oemol())
    pattern_query.BuildExpressions(atomexpr, bondexpr)
    mcss = oechem.OEMCSSearch(pattern_query)
    mcss.SetMCSFunc(oechem.OEMCSMaxAtomsCompleteCycles())

    mcs_results = [match for match in mcss.Match(mol2.to_oemol(), True)]
    print(mcs_results)
    found_mcs = oechem.OEGraphMol()
    oechem.OESubsetMol(found_mcs, mcs_results[0], True)
        
    # Highlight MCS in the molecules
    mol1_highlight = oechem.OEGraphMol()
    oechem.OESubsetMol(mol1_highlight, mol1, mcs_result.GetPattern())
    mol2_highlight = oechem.OEGraphMol()
    oechem.OESubsetMol(mol2_highlight, mol2, mcs_result.GetPattern())

    # Draw the molecules with the highlighted MCS
    opts = oedepict.OE2DMolDisplayOptions(400, 200, oedepict.OEScale_AutoScale)
    disp = oedepict.OE2DMolDisplay(mol1, opts)
    oedepict.OEAddHighlighting(disp, oechem.OEColor(oechem.OEYellow), oechem.OEColor(oechem.OEBlack), mcs_smarts)
    img = oedepict.OEImage(400, 200)
    oedepict.OERenderMolecule(img, disp)

    opts = oedepict.OE2DMolDisplayOptions(400, 200, oedepict.OEScale_AutoScale)
    disp = oedepict.OE2DMolDisplay(mol2, opts)
    oedepict.OEAddHighlighting(disp, oechem.OEColor(oechem.OEYellow), oechem.OEColor(oechem.OEBlack), mcs_smarts)
    img2 = oedepict.OEImage(400, 200)
    oedepict.OERenderMolecule(img2, disp)

    # Display the images
    oedepict.OEWriteImage("molecule1.png", img)
    oedepict.OEWriteImage("molecule2.png", img2)

In [ ]:
find_and_plot_mcss(mol1, mol2)

In [ ]:
mcs_results = [match for match in mcss.Match(mol2.to_oemol(), True)]

In [ ]:
print(mcs_results)

In [ ]:
type(mcs_results[0])

In [ ]:
m = oechem.OEGraphMol()

In [ ]:
match = mcs_results[0]

In [ ]:
match.NumAtoms()

In [ ]:
match.NumBonds()

In [ ]:
match.GetPatternAtoms()

In [ ]:
oechem.OESubsetMol(m, match, True)

In [ ]:
refmol = mols[4].to_oemol()
fitmol = mols[100].to_oemol()

# Find MCS
atomexpr = oechem.OEExprOpts_DefaultAtoms
bondexpr = oechem.OEExprOpts_AutomorphBonds
# create maximum common substructure object


pattern_query = oechem.OEQMol(refmol)
pattern_query.BuildExpressions(atomexpr, bondexpr)
mcss = oechem.OEMCSSearch(pattern_query)
mcss.SetMCSFunc(oechem.OEMCSMaxAtoms())

alignres = oedepict.OEPrepareAlignedDepiction(fitmol, mcss)

image = oedepict.OEImage(400, 200)

rows, cols = 1, 2
grid = oedepict.OEImageGrid(image, rows, cols)

opts = oedepict.OE2DMolDisplayOptions(grid.GetCellWidth(), grid.GetCellHeight(),
                                      oedepict.OEScale_AutoScale)
opts.SetTitleLocation(oedepict.OETitleLocation_Bottom)
opts.SetHydrogenStyle(oedepict.OEHydrogenStyle_Hidden)

refscale = oedepict.OEGetMoleculeScale(refmol, opts)
fitscale = oedepict.OEGetMoleculeScale(fitmol, opts)
opts.SetScale(min(refscale, fitscale))

refdisp = oedepict.OE2DMolDisplay(mcss.GetPattern(), opts)
fitdisp = oedepict.OE2DMolDisplay(fitmol, opts)

if alignres.IsValid():
    refabset = oechem.OEAtomBondSet(alignres.GetPatternAtoms(), alignres.GetPatternBonds())
    oedepict.OEAddHighlighting(refdisp, oechem.OEBlueTint,
                               oedepict.OEHighlightStyle_BallAndStick, refabset)

    fitabset = oechem.OEAtomBondSet(alignres.GetTargetAtoms(), alignres.GetTargetBonds())
    oedepict.OEAddHighlighting(fitdisp, oechem.OEBlueTint,
                               oedepict.OEHighlightStyle_BallAndStick, fitabset)

refcell = grid.GetCell(1, 1)
oedepict.OERenderMolecule(refcell, refdisp)

fitcell = grid.GetCell(1, 2)
oedepict.OERenderMolecule(fitcell, fitdisp)

oedepict.OEWriteImage("MCSAlign.png", image)
Image("MCSAlign.png")

In [ ]:
smiles = [len(mol.smiles) for mol in mols]

In [ ]:
smiles

In [ ]:
mols[4]

In [ ]:
mols[100]

In [ ]:
from asapdiscovery.docking.analysis import calculate_tanimoto_oe, TanimotoType

In [ ]:
calculate_tanimoto_oe(mols[4], mols[100], compute_type=TanimotoType.COLOR)

In [ ]:
calculate_tanimoto_oe(mols[4], mols[100], compute_type=TanimotoType.SHAPE)